In [20]:
# En este archivo definimos las funciones que empleará nuestro sistema de web_scrapping

# Declaramos las librerías que usaremos
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options


# Función que rellena y ejecuta el formulario
def execute_form(fecha_inicio, fecha_fin):
    # Opciones para abrir el navegador a pantalla completa
    #chromeOptions = Options()
    #chromeOptions.add_argument("--kiosk")
    # Declaramos nuestro navegador
    driver = webdriver.Chrome()
    # Abrimos nuestra web
    driver.get('https://www.gw-openscience.org')
    # Seleccionamos en el menú eventos y catálogos
    elem = driver.find_element(By.XPATH, "//a[contains(text(), 'Events and Catalogs')]")   
    # Hacemos Click
    driver.execute_script("arguments[0].click();", elem)
    # Esperamos a que cargue
    driver.implicitly_wait(10)
    # Pulsamos el botón query de la ventana que acaba de cargar
    elem2 = driver.find_element(By.XPATH, "//button[text()='Query']").click() 
    # Esperamos a que se cargue el formulario
    driver.implicitly_wait(10)
    # Vamos a rellenar el formulario
    # Comenzamos por los selectores
    select = Select(driver.find_element(By.ID,'mul-release-sel'))
    # Vamos a seleccionar elementos por su nombre, son las diferentes releases
    select.select_by_visible_text('GWTC-1-marginal')
    select.select_by_visible_text('GWTC-1-confident')
    select.select_by_visible_text('O3_IMBH_marginal')
    select.select_by_visible_text('GWTC-2')
    select.select_by_visible_text('GWTC-2.1-marginal')
    select.select_by_visible_text('GWTC-2.1-confident')
    select.select_by_visible_text('GWTC-3-marginal')
    select.select_by_visible_text('GWTC-3-confident')
    select.select_by_visible_text('Initial_LIGO_Virgo')
    
    # Ahora metemos las fechas desde y hasta
    imputFecha1 = driver.find_element(By.XPATH,"//div[@id='min-datetime-tbox']/input")
    imputFecha1.send_keys(fecha_inicio+'T00:00:00')  
    
    imputFecha2 = driver.find_element(By.XPATH,"//div[@id='max-datetime-tbox']/input")
    imputFecha2.send_keys(fecha_fin+'T00:00:00')  
    
    # Ahora marcamos el checkbox para que nos den la última versión de las detecciones
    # de este modo no tendremos repetidos
    checkbox = driver.find_element(By.XPATH,"//label[@id='version-tbox']/input")
    # Hacemos Click para marcar el checkbox
    driver.execute_script("arguments[0].click();", checkbox)
    
    # Ya tenemos todo, pulsamos el botón submit para generar la tabla con los datos
    # Localizamos el botón
    boton = driver.find_element(By.ID, 'submit-query-btn')
    # Lo pulsamos
    driver.execute_script("arguments[0].click();", boton)
    # Esperamos a que se cargue la tabla
    driver.implicitly_wait(10)
    
    # Cuidado que no selecciona todas las columnas, debemos seleccionar lo que deseamos ver
    # Pulsamos el botón para desplegar el selector de columnas
    desplegable = driver.find_element(By.CLASS_NAME ,'tablesaw-columntoggle-btnwrap').click()
    
    # Ahora ya vemos los selectores de columnas, comprobaremos que estén todos seleccionados
    # pulsando aquellos que no lo estén
    
    # Localizamos el checkbox
    check_version = driver.find_element(By.XPATH,"//label[text()='Version']/input")
    # Comprobamos si está checkeado
    if (check_version.get_attribute("checked") != "true"):
        # Hacemos Click para marcar el checkbox
        driver.execute_script("arguments[0].click();", check_version)
    # Localizamos el checkbox
    check_release = driver.find_element(By.XPATH,"//label[text()='Release']/input")
    # Comprobamos si está checkeado
    if (check_release.get_attribute("checked") != "true"):
        # Hacemos Click para marcar el checkbox
        driver.execute_script("arguments[0].click();", check_release)
    # Localizamos el checkbox
    check_GPS = driver.find_element(By.XPATH,"//label[text()='GPS']/input")
    # Comprobamos si está checkeado
    if (check_GPS.get_attribute("checked") != "true"):
        # Hacemos Click para marcar el checkbox
        driver.execute_script("arguments[0].click();", check_GPS)
    # Localizamos el checkbox
    check_masa1 = driver.find_element(By.XPATH,"//label[text()='Mass 1 (M☉)']/input")
    # Comprobamos si está checkeado
    if (check_masa1.get_attribute("checked") != "true"):
        # Hacemos Click para marcar el checkbox
        driver.execute_script("arguments[0].click();", check_masa1)
    # Localizamos el checkbox
    check_masa2 = driver.find_element(By.XPATH,"//label[text()='Mass 2 (M☉)']/input")
    # Comprobamos si está checkeado
    if (check_masa2.get_attribute("checked") != "true"):
        # Hacemos Click para marcar el checkbox
        driver.execute_script("arguments[0].click();", check_masa2)
    # Localizamos el checkbox
    check_snr = driver.find_element(By.XPATH,"//label[text()='Network SNR']/input")
    # Comprobamos si está checkeado
    if (check_snr.get_attribute("checked") != "true"):
        # Hacemos Click para marcar el checkbox
        driver.execute_script("arguments[0].click();", check_snr)
    # Localizamos el checkbox
    check_distancia = driver.find_element(By.XPATH,"//label[text()='Distance (Mpc)']/input")
    # Comprobamos si está checkeado
    if (check_distancia.get_attribute("checked") != "true"):
        # Hacemos Click para marcar el checkbox
        driver.execute_script("arguments[0].click();", check_distancia)
    # Localizamos el checkbox
    check_xeff = driver.find_element(By.XPATH,"//label[text()='χeff']/input")
    # Comprobamos si está checkeado
    if (check_xeff.get_attribute("checked") != "true"):
        # Hacemos Click para marcar el checkbox
        driver.execute_script("arguments[0].click();", check_xeff)
    # Localizamos el checkbox
    check_masa_total = driver.find_element(By.XPATH,"//label[text()='Total Mass (M☉)']/input")
     # Comprobamos si está checkeado
    if (check_masa_total.get_attribute("checked") != "true"):
        # Hacemos Click para marcar el checkbox
        driver.execute_script("arguments[0].click();", check_masa_total)
    # Localizamos el checkbox
    check_chirp = driver.find_element(By.XPATH,"//label[text()='Chirp Mass (M☉)']/input")
    # Comprobamos si está checkeado
    if (check_chirp.get_attribute("checked") != "true"):
        # Hacemos Click para marcar el checkbox
        driver.execute_script("arguments[0].click();", check_chirp)
    # Localizamos el checkbox
    check_frame = driver.find_element(By.XPATH,"//label[text()='Detector Frame Chirp Mass (M☉)']/input")
    # Comprobamos si está checkeado
    if (check_frame.get_attribute("checked") != "true"):
        # Hacemos Click para marcar el checkbox
        driver.execute_script("arguments[0].click();", check_frame)
    # Localizamos el checkbox
    check_Redshift = driver.find_element(By.XPATH,"//label[text()='Redshift']/input")
    # Comprobamos si está checkeado
    if (check_Redshift.get_attribute("checked") != "true"):
        # Hacemos Click para marcar el checkbox
        driver.execute_script("arguments[0].click();", check_Redshift)
    # Localizamos el checkbox
    check_false_alarm = driver.find_element(By.XPATH,"//label[text()='False Alarm Rate (yr-1)']/input")
    # Comprobamos si está checkeado
    if (check_false_alarm.get_attribute("checked") != "true"):
        # Hacemos Click para marcar el checkbox
        driver.execute_script("arguments[0].click();", check_false_alarm) 
    # Localizamos el checkbox
    check_Pastro = driver.find_element(By.XPATH,"//label[text()='Pastro']/input")
    # Comprobamos si está checkeado
    if (check_Pastro.get_attribute("checked") != "true"):
        # Hacemos Click para marcar el checkbox
        driver.execute_script("arguments[0].click();", check_Pastro) 
    # Localizamos el checkbox
    check_masa_final = driver.find_element(By.XPATH,"//label[text()='Final Mass (M☉)']/input")
    # Comprobamos si está checkeado
    if (check_masa_final.get_attribute("checked") != "true"):
        # Hacemos Click para marcar el checkbox
        driver.execute_script("arguments[0].click();", check_masa_final) 
        
    # Pulsamos el botón para ocultar el desplegable y poder trabajar cómodamente
    desplegable = driver.find_element(By.CLASS_NAME ,'tablesaw-columntoggle-btnwrap').click()
    
    # Ya tenemos la tabla en pantalla, ahora la recorreremos para obtener los datos
    detecciones = driver.find_elements(By.CSS_SELECTOR, "tr")
    return detecciones

execute_form('20150101','20221105')


[<selenium.webdriver.remote.webelement.WebElement (session="0b1c413d7b6c592a8b963308b4c69aca", element="c7350efe-5bc4-4e8b-845c-f88b63323987")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0b1c413d7b6c592a8b963308b4c69aca", element="814b4712-9fcf-4620-a619-c01bcb33b066")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0b1c413d7b6c592a8b963308b4c69aca", element="bcc58909-8034-47f0-9811-62a99fb04537")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0b1c413d7b6c592a8b963308b4c69aca", element="5baf262b-c30f-47ed-be7e-40c4b328bf08")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0b1c413d7b6c592a8b963308b4c69aca", element="7e6794ab-74b3-481f-be3f-5a75cab025a2")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0b1c413d7b6c592a8b963308b4c69aca", element="533ec786-3aa6-403e-994f-233b69b74ce0")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0b1c413d7b6c592a8b963308b4c69aca", element="ab1d8e57-7eb1-443c-a2b9-92